In [1]:
!pip install rouge

Defaulting to user installation because normal site-packages is not writeable
  Using cached rouge-1.0.1-py3-none-any.whl.metadata (4.1 kB)
Using cached rouge-1.0.1-py3-none-any.whl (13 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.1.24-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached grpcio-1.70.0-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.8.0-py3-none-any.whl.metadata (5.8 kB)
  Us

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 3.4 MB/s eta 0:00:01
   ---------------------------------- ----- 1.3/1.5 MB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 3.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# CNN (Convolution Neural Network)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

# Load dataset
data = pd.read_csv('ClinicalData.csv')

# Preprocessing
data.dropna(inplace=True)  
data = data[['Abstract', 'Summarized Abstract']]  

# Split data into training (75%) and test (25%)
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

# Tokenization and padding for text data
tokenizer = Tokenizer(num_words=5000)  
tokenizer.fit_on_texts(train_data['Abstract'])
X_train = tokenizer.texts_to_sequences(train_data['Abstract'])
X_test = tokenizer.texts_to_sequences(test_data['Abstract'])
max_len = 100  
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Tokenize and pad the target data (Summarized Abstract)
target_tokenizer = Tokenizer(num_words=5000)  
target_tokenizer.fit_on_texts(train_data['Summarized Abstract'])
y_train = target_tokenizer.texts_to_sequences(train_data['Summarized Abstract'])
y_test = target_tokenizer.texts_to_sequences(test_data['Summarized Abstract'])
max_summary_len = 50  
y_train = pad_sequences(y_train, maxlen=max_summary_len, padding='post', truncating='post')
y_test = pad_sequences(y_test, maxlen=max_summary_len, padding='post', truncating='post')

# Get the vocabulary size for the output layer
vocabulary_size = len(target_tokenizer.word_index) + 1

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=1, input_length=max_len),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),  
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(50, activation='softmax')
])


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=16)

# Generate predicted summaries for the test data
predicted_summaries = model.predict(X_test)

# Convert sequences back to text
predicted_summaries_text = [target_tokenizer.sequences_to_texts([seq])[0] for seq in predicted_summaries]
actual_summaries_text = [target_tokenizer.sequences_to_texts([seq])[0] for seq in y_test]

from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

# Convert sequences back to text for references and hypotheses
references = [summary.split() for summary in actual_summaries_text]
hypotheses = [summary.split() for summary in predicted_summaries_text]

# Calculate ROUGE scores
rouge_1 = corpus_bleu(references, hypotheses, weights=(1, 0, 0, 0))
rouge_2 = corpus_bleu(references, hypotheses, weights=(0, 1, 0, 0))
rouge_l = corpus_bleu(references, hypotheses, weights=(0, 0, 1, 0))
rouge_lsum = corpus_bleu(references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25))

print("ROUGE-1:", rouge_1)
print("ROUGE-2:", rouge_2)
print("ROUGE-L:", rouge_l)
print("ROUGE-Lsum:", rouge_lsum)



C:\Users\sonal\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - loss: 59799.7500
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 60077.1406 
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 60190.6562
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 58137.8086
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 59403.2539
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
ROUGE-1: 0
ROUGE-2: 0
ROUGE-L: 0
ROUGE-Lsum: 0


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load dataset
data = pd.read_csv('ClinicalData.csv')

# Preprocessing
data.dropna(inplace=True)  
data = data[['Abstract', 'Summarized Abstract']]  

# Split data into training (75%) and test (25%)
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

# Tokenization and padding for text data
tokenizer = Tokenizer(num_words=5000)  
tokenizer.fit_on_texts(train_data['Abstract'])
X_train = tokenizer.texts_to_sequences(train_data['Abstract'])
X_test = tokenizer.texts_to_sequences(test_data['Abstract'])
max_len = 100  
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Tokenize and pad the target data (Summarized Abstract)
target_tokenizer = Tokenizer(num_words=5000)  
target_tokenizer.fit_on_texts(train_data['Summarized Abstract'])
y_train = target_tokenizer.texts_to_sequences(train_data['Summarized Abstract'])
y_test = target_tokenizer.texts_to_sequences(test_data['Summarized Abstract'])
max_summary_len = 50  
y_train = pad_sequences(y_train, maxlen=max_summary_len, padding='post', truncating='post')
y_test = pad_sequences(y_test, maxlen=max_summary_len, padding='post', truncating='post')

# Build a sequence-to-sequence CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_len),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.RepeatVector(max_summary_len),  # Match encoder and decoder lengths
    tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),  # Added dropout
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(target_tokenizer.word_index) + 1, activation='softmax'))
])

# Compile the model with 'sparse_categorical_crossentropy' loss
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy')  # Adjusted learning rate

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=16)  

# Generate predicted summaries for the test data
predicted_summaries = model.predict(X_test)


# Implement beam search
def beam_search_decoder(data, k):
    sequences = [[list(), 1.0]]
    for row in data:
        all_candidates = list()
        for i in range(len(sequences)):
            seq, score = sequences[i]
            for j in range(len(row)):
                candidate = [seq + [j], score * -np.log(row[j])]
                all_candidates.append(candidate)
        ordered = sorted(all_candidates, key=lambda tup: tup[1])
        sequences = ordered[:k]
    return sequences

# Perform beam search for each example in the test data
beam_search_results = [beam_search_decoder(sample, k=3) for sample in predicted_summaries]

# Convert sequences back to text and print examples
for i in range(10):  
    actual_summary = " ".join([target_tokenizer.index_word.get(word, "") for word in y_test[i] if word != 0])
    predicted_summaries_text = []
    for beam_result in beam_search_results[i]:
        predicted_summary = " ".join([target_tokenizer.index_word.get(word, "") for word in beam_result[0] if word != 0])
        predicted_summaries_text.append(predicted_summary)
    print(f"Example - Actual Summary: {actual_summary}")
    print(f"Example - Predicted Summaries: {predicted_summaries_text}\n")


C:\Users\sonal\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 157ms/step - loss: 7.1071
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 7.0780
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - loss: 6.9940
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 6.8148
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 6.6053
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 6.4102
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - loss: 6.2280
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - loss: 6.1416
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 6.1174
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - loss: 6.0606
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Example - Actual Summary: are the primary for the of and and models have in understanding these however the of a such as a should factors this scoping review aims to identify and the most factors as well as for of and
Example - Predicted Summaries: ['the the the the the the the the the the the the the the the t

RNN (Recurrent Neural Network)

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

# Load dataset
data = pd.read_csv('ClinicalData.csv')

# Preprocessing
data.dropna(inplace=True)  
data = data[['Abstract', 'Summarized Abstract']]  

# Split data into training (75%) and test (25%)
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

# Tokenization and padding for text data
tokenizer = Tokenizer(num_words=5000)  
tokenizer.fit_on_texts(train_data['Abstract'])
X_train = tokenizer.texts_to_sequences(train_data['Abstract'])
X_test = tokenizer.texts_to_sequences(test_data['Abstract'])
max_len = 100  
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Tokenize and pad the target data (Summarized Abstract)
target_tokenizer = Tokenizer(num_words=5000)  
target_tokenizer.fit_on_texts(train_data['Summarized Abstract'])
y_train = target_tokenizer.texts_to_sequences(train_data['Summarized Abstract'])
y_test = target_tokenizer.texts_to_sequences(test_data['Summarized Abstract'])
max_summary_len = 100  
y_train = pad_sequences(y_train, maxlen=max_summary_len, padding='post', truncating='post')
y_test = pad_sequences(y_test, maxlen=max_summary_len, padding='post', truncating='post')

# Build and compile the RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=100, input_length=max_len),
    tf.keras.layers.SimpleRNN(256,  activation='elu',  return_sequences=True),
    tf.keras.layers.SimpleRNN(256,  activation='elu',  return_sequences=True),
    tf.keras.layers.SimpleRNN(256,  activation='elu',  return_sequences=True),
    tf.keras.layers.SimpleRNN(256,  activation='elu',  return_sequences=True),
    tf.keras.layers.SimpleRNN(256,  activation='elu',  return_sequences=True),

    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5000, activation='softmax'))
])


# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=16)

# Generate predicted summaries for the test data
predicted_summaries = model.predict(X_test)

# Convert sequences back to text
predicted_summaries_text = []
for seq in predicted_summaries:
    text_seq = [str(word) for word in seq]
    text_seq = ' '.join(text_seq).strip()
    predicted_summaries_text.append(text_seq)

actual_summaries_text = []
for seq in y_test:
    text_seq = [str(word) for word in seq]
    text_seq = ' '.join(text_seq).strip()
    actual_summaries_text.append(text_seq)


# Calculate ROUGE scores
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

# Calculate ROUGE scores
rouge_1 = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(1, 0, 0, 0))
rouge_2 = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0, 1, 0, 0))
rouge_l = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0, 0, 1, 0))
rouge_lsum = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0.25, 0.25, 0.25, 0.25))

print("ROUGE-1:", rouge_1)
print("ROUGE-2:", rouge_2)
print("ROUGE-L:", rouge_l)
print("ROUGE-Lsum:", rouge_lsum)


C:\Users\sonal\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 25s 743ms/step - accuracy: 0.0224 - loss: 8.4529
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 762ms/step - accuracy: 0.0843 - loss: 8.8369
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 631ms/step - accuracy: 0.1914 - loss: 6.2584
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 683ms/step - accuracy: 0.1961 - loss: 5.7790
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 599ms/step - accuracy: 0.1980 - loss: 5.5767
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
ROUGE-1: 0
ROUGE-2: 0
ROUGE-L: 0
ROUGE-Lsum: 0


In [4]:
# RNN print actual summary and predicted summary

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load dataset
data = pd.read_csv('ClinicalData.csv')

# Preprocessing
data.dropna(inplace=True)  
data = data[['Abstract', 'Summarized Abstract']]  

# Split data into training (75%) and test (25%)
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

# Tokenization and padding for text data
tokenizer = Tokenizer(num_words=5000)  
tokenizer.fit_on_texts(train_data['Abstract'])
X_train = tokenizer.texts_to_sequences(train_data['Abstract'])
X_test = tokenizer.texts_to_sequences(test_data['Abstract'])
max_len = 100  
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Tokenize and pad the target data (Summarized Abstract)
target_tokenizer = Tokenizer(num_words=5000)  
target_tokenizer.fit_on_texts(train_data['Summarized Abstract'])
y_train = target_tokenizer.texts_to_sequences(train_data['Summarized Abstract'])
y_test = target_tokenizer.texts_to_sequences(test_data['Summarized Abstract'])
max_summary_len = 100  
y_train = pad_sequences(y_train, maxlen=max_summary_len, padding='post', truncating='post')
y_test = pad_sequences(y_test, maxlen=max_summary_len, padding='post', truncating='post')

# Build and compile the RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=100, input_length=max_len),
    tf.keras.layers.SimpleRNN(256, activation='elu', return_sequences=True),
    tf.keras.layers.SimpleRNN(256, activation='elu', return_sequences=True),
    tf.keras.layers.SimpleRNN(256, activation='elu', return_sequences=True),
    tf.keras.layers.SimpleRNN(256, activation='elu', return_sequences=True),
    tf.keras.layers.SimpleRNN(256, activation='elu', return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5000, activation='softmax'))
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=16)

# Generate predicted summaries for the test data
predicted_summaries = model.predict(X_test)

# Implement beam search
def beam_search_decoder(data, k):
    sequences = [[list(), 1.0]]
    for row in data:
        all_candidates = list()
        for i in range(len(sequences)):
            seq, score = sequences[i]
            for j in range(len(row)):
                candidate = [seq + [j], score * -np.log(row[j])]
                all_candidates.append(candidate)
        ordered = sorted(all_candidates, key=lambda tup: tup[1])
        sequences = ordered[:k]
    return sequences

# Perform beam search for each example in the test data
beam_search_results = [beam_search_decoder(sample, k=3) for sample in predicted_summaries]

# Convert sequences back to text and print examples
for i in range(10):  
    actual_summary = " ".join([target_tokenizer.index_word.get(word, "") for word in y_test[i] if word != 0])
    predicted_summaries_text = []
    for beam_result in beam_search_results[i]:
        predicted_summary = " ".join([target_tokenizer.index_word.get(word, "") for word in beam_result[0] if word != 0])
        predicted_summaries_text.append(predicted_summary)
    print(f"Example - Actual Summary: {actual_summary}")
    print(f"Example - Predicted Summaries: {predicted_summaries_text}\n")

C:\Users\sonal\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 20s 556ms/step - accuracy: 0.0230 - loss: 8.4212
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 547ms/step - accuracy: 0.1891 - loss: 9.0500
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 546ms/step - accuracy: 0.0599 - loss: 6.4994
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 546ms/step - accuracy: 0.1242 - loss: 5.7341
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 568ms/step - accuracy: 0.1951 - loss: 5.4709
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


C:\Users\sonal\AppData\Local\Temp\ipykernel_20964\320236236.py:66: RuntimeWarning: overflow encountered in scalar multiply
  candidate = [seq + [j], score * -np.log(row[j])]


Example - Actual Summary: are the primary for the of and and models have in understanding these however the of a such as a should factors this scoping review aims to identify and the most factors as well as for of and
Example - Predicted Summaries: ['and of to for of systems in to of', 'and of to for of systems in to of the', 'and of to for of systems in to of of']

Example - Actual Summary: the of clinical is a before more natural language processing models that high accuracy for 1 experience a large of accuracy when to the of this study is to develop methods that clinical the with improved we found improved models only when with in samples improving the score from 0 to 0
Example - Predicted Summaries: ['', 'in', 'with']

Example - Actual Summary: the prevention of for patients with is still a great in clinical practice there are studies that to search for strategies to the and life for these patients we aim to the efficacy between different reported treatments by meta analysis most o

# LSTM (Long short-term memory)

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

# Load dataset
data = pd.read_csv('ClinicalData.csv')

# Preprocessing
data.dropna(inplace=True)  
data = data[['Abstract', 'Summarized Abstract']]  

# Split data into training (75%) and test (25%)
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

# Tokenization and padding for text data
tokenizer = Tokenizer(num_words=5000)  
tokenizer.fit_on_texts(train_data['Abstract'])
X_train = tokenizer.texts_to_sequences(train_data['Abstract'])
X_test = tokenizer.texts_to_sequences(test_data['Abstract'])
max_len = 100  
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Tokenize and pad the target data (Summarized Abstract)
target_tokenizer = Tokenizer(num_words=5000)  
target_tokenizer.fit_on_texts(train_data['Summarized Abstract'])
y_train = target_tokenizer.texts_to_sequences(train_data['Summarized Abstract'])
y_test = target_tokenizer.texts_to_sequences(test_data['Summarized Abstract'])
max_summary_len = 100  
y_train = pad_sequences(y_train, maxlen=max_summary_len, padding='post', truncating='post')
y_test = pad_sequences(y_test, maxlen=max_summary_len, padding='post', truncating='post')

# Build and compile the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=100, input_length=max_len),
    tf.keras.layers.LSTM(256, return_sequences=True, activation='relu'),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5000, activation='softmax'))
])


# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=16)

# Generate predicted summaries for the test data
predicted_summaries = model.predict(X_test)

# Convert sequences back to text
predicted_summaries_text = []
for seq in predicted_summaries:
    text_seq = [str(word) for word in seq]
    text_seq = ' '.join(text_seq).strip()
    predicted_summaries_text.append(text_seq)

actual_summaries_text = []
for seq in y_test:
    text_seq = [str(word) for word in seq]
    text_seq = ' '.join(text_seq).strip()
    actual_summaries_text.append(text_seq)


# Calculate ROUGE scores
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

# Calculate ROUGE scores
rouge_1 = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(1, 0, 0, 0))
rouge_2 = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0, 1, 0, 0))
rouge_l = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0, 0, 1, 0))
rouge_lsum = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0.25, 0.25, 0.25, 0.25))

print("ROUGE-1:", rouge_1)
print("ROUGE-2:", rouge_2)
print("ROUGE-L:", rouge_l)
print("ROUGE-Lsum:", rouge_lsum)

Epoch 1/5


C:\Users\sonal\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 17s 678ms/step - accuracy: 0.0133 - loss: 8.5163
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 615ms/step - accuracy: 0.1857 - loss: 8.5045
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 563ms/step - accuracy: 0.1852 - loss: 8.4692
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 672ms/step - accuracy: 0.1896 - loss: 7.5973
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 629ms/step - accuracy: 0.1884 - loss: 8.4098
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
ROUGE-1: 0
ROUGE-2: 0
ROUGE-L: 0
ROUGE-Lsum: 0


In [6]:
#LSTM - PRINTING ACTUAL AND PREDICTED SUMMARIES TEXT
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

# Load dataset
data = pd.read_csv('ClinicalData.csv')

# Preprocessing
data.dropna(inplace=True)  
data = data[['Abstract', 'Summarized Abstract']]  

# Split data into training (75%) and test (25%)
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

# Tokenization and padding for text data
tokenizer = Tokenizer(num_words=5000)  
tokenizer.fit_on_texts(train_data['Abstract'])
X_train = tokenizer.texts_to_sequences(train_data['Abstract'])
X_test = tokenizer.texts_to_sequences(test_data['Abstract'])
max_len = 100  
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Tokenize and pad the target data (Summarized Abstract)
target_tokenizer = Tokenizer(num_words=5000) 
target_tokenizer.fit_on_texts(train_data['Summarized Abstract'])
y_train = target_tokenizer.texts_to_sequences(train_data['Summarized Abstract'])
y_test = target_tokenizer.texts_to_sequences(test_data['Summarized Abstract'])
max_summary_len = 100  
y_train = pad_sequences(y_train, maxlen=max_summary_len, padding='post', truncating='post')
y_test = pad_sequences(y_test, maxlen=max_summary_len, padding='post', truncating='post')

# Build and compile the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=100, input_length=max_len),
    tf.keras.layers.LSTM(256, return_sequences=True, activation='relu'),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5000, activation='softmax'))
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=16)

# Generate predicted summaries for the test data
predicted_summaries = model.predict(X_test)

# Implement beam search
def beam_search_decoder(data, k):
    sequences = [[list(), 1.0]]
    for row in data:
        all_candidates = list()
        for i in range(len(sequences)):
            seq, score = sequences[i]
            for j in range(len(row)):
                candidate = [seq + [j], score * -np.log(row[j])]
                all_candidates.append(candidate)
        ordered = sorted(all_candidates, key=lambda tup: tup[1])
        sequences = ordered[:k]
    return sequences

# Perform beam search for each example in the test data
beam_search_results = [beam_search_decoder(sample, k=3) for sample in predicted_summaries]

# Convert sequences back to text and print examples
for i in range(10):  
    actual_summary = " ".join([target_tokenizer.index_word.get(word, "") for word in y_test[i] if word != 0])
    predicted_summaries_text = []
    for beam_result in beam_search_results[i]:
        predicted_summary = " ".join([target_tokenizer.index_word.get(word, "") for word in beam_result[0] if word != 0])
        predicted_summaries_text.append(predicted_summary)
    print(f"Example - Actual Summary: {actual_summary}")
    print(f"Example - Predicted Summaries: {predicted_summaries_text}\n")
     

Epoch 1/5


C:\Users\sonal\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 18s 638ms/step - accuracy: 0.0680 - loss: 8.5152
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 579ms/step - accuracy: 0.1814 - loss: 8.5026
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 616ms/step - accuracy: 0.1822 - loss: 8.4475
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 580ms/step - accuracy: 0.1943 - loss: 7.6132
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 615ms/step - accuracy: 0.1547 - loss: 8.1312
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


C:\Users\sonal\AppData\Local\Temp\ipykernel_20964\2343647717.py:62: RuntimeWarning: divide by zero encountered in log
  candidate = [seq + [j], score * -np.log(row[j])]
C:\Users\sonal\AppData\Local\Temp\ipykernel_20964\2343647717.py:62: RuntimeWarning: invalid value encountered in scalar multiply
  candidate = [seq + [j], score * -np.log(row[j])]


Example - Actual Summary: are the primary for the of and and models have in understanding these however the of a such as a should factors this scoping review aims to identify and the most factors as well as for of and
Example - Predicted Summaries: ['and', 'and the', 'and of']

Example - Actual Summary: the of clinical is a before more natural language processing models that high accuracy for 1 experience a large of accuracy when to the of this study is to develop methods that clinical the with improved we found improved models only when with in samples improving the score from 0 to 0
Example - Predicted Summaries: ['the', 'the the', 'the of']

Example - Actual Summary: the prevention of for patients with is still a great in clinical practice there are studies that to search for strategies to the and life for these patients we aim to the efficacy between different reported treatments by meta analysis most of the studies were high quality with low bias and were when the of life
Example 

# CNN + RNN

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

# Load dataset
data = pd.read_csv('ClinicalData.csv')

# Preprocessing
data.dropna(inplace=True)  
data = data[['Abstract', 'Summarized Abstract']]  

# Split data into training (75%) and test (25%)
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

# Tokenization and padding for text data
tokenizer = Tokenizer(num_words=5000)  
tokenizer.fit_on_texts(train_data['Abstract'])
X_train = tokenizer.texts_to_sequences(train_data['Abstract'])
X_test = tokenizer.texts_to_sequences(test_data['Abstract'])
max_len = 100  
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Tokenize and pad the target data (Summarized Abstract)
target_tokenizer = Tokenizer(num_words=5000)  
target_tokenizer.fit_on_texts(train_data['Summarized Abstract'])
y_train = target_tokenizer.texts_to_sequences(train_data['Summarized Abstract'])
y_test = target_tokenizer.texts_to_sequences(test_data['Summarized Abstract'])
max_summary_len = 48  

# Padding or truncating the input sequences
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Padding or truncating the target sequences
y_train = pad_sequences(y_train, maxlen=max_summary_len, padding='post', truncating='post')
y_test = pad_sequences(y_test, maxlen=max_summary_len, padding='post', truncating='post')


# Build and compile the CNN-RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=100, input_length=max_len),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),  
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.LSTM(256, activation='relu', return_sequences=True),
    tf.keras.layers.LSTM(256, activation='relu', return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5000, activation='softmax'))
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=16)

# Generate predicted summaries for the test data
predicted_summaries = model.predict(X_test)

# Convert sequences back to text
predicted_summaries_text = []
for seq in predicted_summaries:
    text_seq = [str(word) for word in seq]
    text_seq = ' '.join(text_seq).strip()
    predicted_summaries_text.append(text_seq)

actual_summaries_text = []
for seq in y_test:
    text_seq = [str(word) for word in seq]
    text_seq = ' '.join(text_seq).strip()
    actual_summaries_text.append(text_seq)

# Calculate ROUGE scores
from nltk.translate.bleu_score import corpus_bleu

# Calculate ROUGE scores
rouge_1 = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(1, 0, 0, 0))
rouge_2 = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0, 1, 0, 0))
rouge_l = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0, 0, 1, 0))
rouge_lsum = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0.25, 0.25, 0.25, 0.25))

print("ROUGE-1:", rouge_1)
print("ROUGE-2:", rouge_2)
print("ROUGE-L:", rouge_l)
print("ROUGE-Lsum:", rouge_lsum)

C:\Users\sonal\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 15s 404ms/step - accuracy: 0.0199 - loss: 8.5164
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 465ms/step - accuracy: 0.0464 - loss: 8.5055
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 413ms/step - accuracy: 0.0469 - loss: 8.4322
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 351ms/step - accuracy: 0.0438 - loss: 8.9490
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 357ms/step - accuracy: 0.0495 - loss: 8.2497
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
ROUGE-1: 0
ROUGE-2: 0
ROUGE-L: 0
ROUGE-Lsum: 0


In [8]:
#cnn + rnn - PRINTING ACTUAL AND PREDICTED SUMMARIES
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

# Load dataset
data = pd.read_csv('ClinicalData.csv')

# Preprocessing
data.dropna(inplace=True)  
data = data[['Abstract', 'Summarized Abstract']]  

# Split data into training (75%) and test (25%)
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

# Tokenization and padding for text data
tokenizer = Tokenizer(num_words=5000)  
tokenizer.fit_on_texts(train_data['Abstract'])
X_train = tokenizer.texts_to_sequences(train_data['Abstract'])
X_test = tokenizer.texts_to_sequences(test_data['Abstract'])
max_len = 100  
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Tokenize and pad the target data (Summarized Abstract)
target_tokenizer = Tokenizer(num_words=5000)  
target_tokenizer.fit_on_texts(train_data['Summarized Abstract'])
y_train = target_tokenizer.texts_to_sequences(train_data['Summarized Abstract'])
y_test = target_tokenizer.texts_to_sequences(test_data['Summarized Abstract'])
max_summary_len = 48  

# Padding or truncating the input sequences
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Padding or truncating the target sequences
y_train = pad_sequences(y_train, maxlen=max_summary_len, padding='post', truncating='post')
y_test = pad_sequences(y_test, maxlen=max_summary_len, padding='post', truncating='post')


# Build and compile the CNN-RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=100, input_length=max_len),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),  
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.LSTM(256, activation='relu', return_sequences=True),
    tf.keras.layers.LSTM(256, activation='relu', return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5000, activation='softmax'))
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=16)

# Generate predicted summaries for the test data
predicted_summaries = model.predict(X_test)

# Implement beam search
def beam_search_decoder(data, k):
    sequences = [[list(), 1.0]]
    for row in data:
        all_candidates = list()
        for i in range(len(sequences)):
            seq, score = sequences[i]
            for j in range(len(row)):
                candidate = [seq + [j], score * -np.log(row[j])]
                all_candidates.append(candidate)
        ordered = sorted(all_candidates, key=lambda tup: tup[1])
        sequences = ordered[:k]
    return sequences

# Perform beam search for each example in the test data
beam_search_results = [beam_search_decoder(sample, k=3) for sample in predicted_summaries]

# Convert sequences back to text and print examples
for i in range(10):  
    actual_summary = " ".join([target_tokenizer.index_word.get(word, "") for word in y_test[i] if word != 0])
    predicted_summaries_text = []
    for beam_result in beam_search_results[i]:
        predicted_summary = " ".join([target_tokenizer.index_word.get(word, "") for word in beam_result[0] if word != 0])
        predicted_summaries_text.append(predicted_summary)
    print(f"Example - Actual Summary: {actual_summary}")
    print(f"Example - Predicted Summaries: {predicted_summaries_text}\n")

Epoch 1/5


C:\Users\sonal\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 413ms/step - accuracy: 0.0026 - loss: 8.5166 
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 337ms/step - accuracy: 0.0480 - loss: 8.5070
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 401ms/step - accuracy: 0.0487 - loss: 8.4574
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 317ms/step - accuracy: 0.0463 - loss: 8.2681
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 409ms/step - accuracy: 0.0481 - loss: 8.3496
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


C:\Users\sonal\AppData\Local\Temp\ipykernel_20964\3102310156.py:72: RuntimeWarning: overflow encountered in scalar multiply
  candidate = [seq + [j], score * -np.log(row[j])]


Example - Actual Summary: are the primary for the of and and models have in understanding these however the of a such as a should factors this scoping review aims to identify and the most factors as well as for of and
Example - Predicted Summaries: ['of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of', 'of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of the', 'of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of']

Example - Actual Summary: the of clinical is a before more natural language processing models that high accuracy for 1 experience a large of accuracy when to the of this study is to develop methods that clinical the with improved we found improved models only when with in samples improving the score
Example - Predicted Summaries: ['of of of of of of of 

# biLSTM (Bidirectional LSTM)

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

# Load dataset
data = pd.read_csv('ClinicalData.csv')

# Preprocessing
data.dropna(inplace=True) 
data = data[['Abstract', 'Summarized Abstract']]  

# Split data into training (75%) and test (25%)
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

# Tokenization and padding for text data
tokenizer = Tokenizer(num_words=5000)  
tokenizer.fit_on_texts(train_data['Abstract'])
X_train = tokenizer.texts_to_sequences(train_data['Abstract'])
X_test = tokenizer.texts_to_sequences(test_data['Abstract'])
max_len = 100  
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Tokenize and pad the target data (Summarized Abstract)
target_tokenizer = Tokenizer(num_words=5000)  
target_tokenizer.fit_on_texts(train_data['Summarized Abstract'])
y_train = target_tokenizer.texts_to_sequences(train_data['Summarized Abstract'])
y_test = target_tokenizer.texts_to_sequences(test_data['Summarized Abstract'])
max_summary_len = 48  

# Padding or truncating the input sequences
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Padding or truncating the target sequences
y_train = pad_sequences(y_train, maxlen=max_summary_len, padding='post', truncating='post')
y_test = pad_sequences(y_test, maxlen=max_summary_len, padding='post', truncating='post')

# Build and compile the BiLSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=100, input_length=max_len),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),  
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, activation='relu', return_sequences=True)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5000, activation='softmax'))
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=16)

# Generate predicted summaries for the test data
predicted_summaries = model.predict(X_test)

# Convert sequences back to text
predicted_summaries_text = []
for seq in predicted_summaries:
    text_seq = [str(word) for word in seq]
    text_seq = ' '.join(text_seq).strip()
    predicted_summaries_text.append(text_seq)

actual_summaries_text = []
for seq in y_test:
    text_seq = [str(word) for word in seq]
    text_seq = ' '.join(text_seq).strip()
    actual_summaries_text.append(text_seq)

# Calculate ROUGE scores
from nltk.translate.bleu_score import corpus_bleu

# Calculate ROUGE scores
rouge_1 = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(1, 0, 0, 0))
rouge_2 = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0, 1, 0, 0))
rouge_l = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0, 0, 1, 0))
rouge_lsum = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0.25, 0.25, 0.25, 0.25))

print("ROUGE-1:", rouge_1)
print("ROUGE-2:", rouge_2)
print("ROUGE-L:", rouge_l)
print("ROUGE-Lsum:", rouge_lsum)

Epoch 1/5


C:\Users\sonal\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 16s 481ms/step - accuracy: 0.0026 - loss: 8.5161 
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 496ms/step - accuracy: 0.0518 - loss: 8.4960
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 611ms/step - accuracy: 0.0490 - loss: 8.4195
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 480ms/step - accuracy: 0.0485 - loss: 8.6334
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 460ms/step - accuracy: 0.0448 - loss: 7.9795
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
ROUGE-1: 0
ROUGE-2: 0
ROUGE-L: 0
ROUGE-Lsum: 0


In [10]:
#bilstm - PRINTING ACTUAL AND PREDICTED SUMMARIES TEXT
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

# Load dataset
data = pd.read_csv('ClinicalData.csv')

# Preprocessing
data.dropna(inplace=True)  
data = data[['Abstract', 'Summarized Abstract']]  

# Split data into training (75%) and test (25%)
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

# Tokenization and padding for text data
tokenizer = Tokenizer(num_words=5000)  
tokenizer.fit_on_texts(train_data['Abstract'])
X_train = tokenizer.texts_to_sequences(train_data['Abstract'])
X_test = tokenizer.texts_to_sequences(test_data['Abstract'])
max_len = 100  
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Tokenize and pad the target data (Summarized Abstract)
target_tokenizer = Tokenizer(num_words=5000)  
target_tokenizer.fit_on_texts(train_data['Summarized Abstract'])
y_train = target_tokenizer.texts_to_sequences(train_data['Summarized Abstract'])
y_test = target_tokenizer.texts_to_sequences(test_data['Summarized Abstract'])
max_summary_len = 48  

# Padding or truncating the input sequences
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Padding or truncating the target sequences
y_train = pad_sequences(y_train, maxlen=max_summary_len, padding='post', truncating='post')
y_test = pad_sequences(y_test, maxlen=max_summary_len, padding='post', truncating='post')

# Build and compile the BiLSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=100, input_length=max_len),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),  # You can adjust the filter size and number of filters
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, activation='relu', return_sequences=True)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5000, activation='softmax'))
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=16)

predicted_summaries = model.predict(X_test)

# Implement beam search
def beam_search_decoder(data, k):
    sequences = [[list(), 1.0]]
    for row in data:
        all_candidates = list()
        for i in range(len(sequences)):
            seq, score = sequences[i]
            for j in range(len(row)):
                candidate = [seq + [j], score * -np.log(row[j])]
                all_candidates.append(candidate)
        ordered = sorted(all_candidates, key=lambda tup: tup[1])
        sequences = ordered[:k]
    return sequences

# Perform beam search for each example in the test data
beam_search_results = [beam_search_decoder(sample, k=3) for sample in predicted_summaries]

# Convert sequences back to text and print examples
for i in range(10):  
    actual_summary = " ".join([target_tokenizer.index_word.get(word, "") for word in y_test[i] if word != 0])
    predicted_summaries_text = []
    for beam_result in beam_search_results[i]:
        predicted_summary = " ".join([target_tokenizer.index_word.get(word, "") for word in beam_result[0] if word != 0])
        predicted_summaries_text.append(predicted_summary)
    print(f"Example - Actual Summary: {actual_summary}")
    print(f"Example - Predicted Summaries: {predicted_summaries_text}\n")

Epoch 1/5


C:\Users\sonal\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 444ms/step - accuracy: 0.0182 - loss: 8.5152
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 447ms/step - accuracy: 0.0474 - loss: 8.4898
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 494ms/step - accuracy: 0.0510 - loss: 8.3576
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 444ms/step - accuracy: 0.0477 - loss: 9.1156
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 451ms/step - accuracy: 0.0506 - loss: 8.1282
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


C:\Users\sonal\AppData\Local\Temp\ipykernel_20964\1869521072.py:69: RuntimeWarning: overflow encountered in scalar multiply
  candidate = [seq + [j], score * -np.log(row[j])]


Example - Actual Summary: are the primary for the of and and models have in understanding these however the of a such as a should factors this scoping review aims to identify and the most factors as well as for of and
Example - Predicted Summaries: ['of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of', 'of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of the', 'of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of']

Example - Actual Summary: the of clinical is a before more natural language processing models that high accuracy for 1 experience a large of accuracy when to the of this study is to develop methods that clinical the with improved we found improved models only when with in samples improving the score
Example - Predicted Summaries: ['of of of of of of of 

# CNN + LSTM

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

# Load your dataset
data = pd.read_csv('ClinicalData.csv')

# Preprocessing
data.dropna(inplace=True)  
data = data[['Abstract', 'Summarized Abstract']]  

# Split data into training (75%) and test (25%)
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

# Tokenization and padding for text data
tokenizer = Tokenizer(num_words=5000)  
tokenizer.fit_on_texts(train_data['Abstract'])
X_train = tokenizer.texts_to_sequences(train_data['Abstract'])
X_test = tokenizer.texts_to_sequences(test_data['Abstract'])
max_len = 100  
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Tokenize and pad the target data (Summarized Abstract)
target_tokenizer = Tokenizer(num_words=5000)  
target_tokenizer.fit_on_texts(train_data['Summarized Abstract'])
y_train = target_tokenizer.texts_to_sequences(train_data['Summarized Abstract'])
y_test = target_tokenizer.texts_to_sequences(test_data['Summarized Abstract'])
max_summary_len = 48  

# Padding or truncating the input sequences
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Padding or truncating the target sequences
y_train = pad_sequences(y_train, maxlen=max_summary_len, padding='post', truncating='post')
y_test = pad_sequences(y_test, maxlen=max_summary_len, padding='post', truncating='post')

# Build and compile the CNN + LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=100, input_length=max_len),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),  
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.LSTM(256, activation='relu', return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5000, activation='softmax'))
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=16)

# Generate predicted summaries for the test data
predicted_summaries = model.predict(X_test)

# Convert sequences back to text
predicted_summaries_text = []
for seq in predicted_summaries:
    text_seq = [str(word) for word in seq]
    text_seq = ' '.join(text_seq).strip()
    predicted_summaries_text.append(text_seq)

actual_summaries_text = []
for seq in y_test:
    text_seq = [str(word) for word in seq]
    text_seq = ' '.join(text_seq).strip()
    actual_summaries_text.append(text_seq)

# Calculate ROUGE scores
from nltk.translate.bleu_score import corpus_bleu

# Calculate ROUGE scores
rouge_1 = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(1, 0, 0, 0))
rouge_2 = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0, 1, 0, 0))
rouge_l = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0, 0, 1, 0))
rouge_lsum = corpus_bleu([actual_summaries_text], [predicted_summaries_text], weights=(0.25, 0.25, 0.25, 0.25))

print("ROUGE-1:", rouge_1)
print("ROUGE-2:", rouge_2)
print("ROUGE-L:", rouge_l)
print("ROUGE-Lsum:", rouge_lsum)

C:\Users\sonal\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 330ms/step - accuracy: 0.0020 - loss: 8.5162 
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 300ms/step - accuracy: 0.0488 - loss: 8.5017
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 283ms/step - accuracy: 0.0446 - loss: 8.4331
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 290ms/step - accuracy: 0.0487 - loss: 9.4214
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 280ms/step - accuracy: 0.0348 - loss: 8.0365
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
ROUGE-1: 0
ROUGE-2: 0
ROUGE-L: 0
ROUGE-Lsum: 0


In [12]:
#cnn+lstm PRINTING ACTUAL AND PREDICTED SUMMARIES

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

# Load dataset
data = pd.read_csv('ClinicalData.csv')

# Preprocessing
data.dropna(inplace=True)  
data = data[['Abstract', 'Summarized Abstract']]  

# Split data into training (75%) and test (25%)
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

# Tokenization and padding for text data
tokenizer = Tokenizer(num_words=5000)  
tokenizer.fit_on_texts(train_data['Abstract'])
X_train = tokenizer.texts_to_sequences(train_data['Abstract'])
X_test = tokenizer.texts_to_sequences(test_data['Abstract'])
max_len = 100  
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Tokenize and pad the target data (Summarized Abstract)
target_tokenizer = Tokenizer(num_words=5000)  
target_tokenizer.fit_on_texts(train_data['Summarized Abstract'])
y_train = target_tokenizer.texts_to_sequences(train_data['Summarized Abstract'])
y_test = target_tokenizer.texts_to_sequences(test_data['Summarized Abstract'])
max_summary_len = 48  

# Padding or truncating the input sequences
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

# Padding or truncating the target sequences
y_train = pad_sequences(y_train, maxlen=max_summary_len, padding='post', truncating='post')
y_test = pad_sequences(y_test, maxlen=max_summary_len, padding='post', truncating='post')

# Build and compile the CNN + LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=100, input_length=max_len),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),  
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.LSTM(256, activation='relu', return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5000, activation='softmax'))
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=16)

# Generate predicted summaries for the test data
predicted_summaries = model.predict(X_test)

# Implement beam search
def beam_search_decoder(data, k):
    sequences = [[list(), 1.0]]
    for row in data:
        all_candidates = list()
        for i in range(len(sequences)):
            seq, score = sequences[i]
            for j in range(len(row)):
                candidate = [seq + [j], score * -np.log(row[j])]
                all_candidates.append(candidate)
        ordered = sorted(all_candidates, key=lambda tup: tup[1])
        sequences = ordered[:k]
    return sequences

# Perform beam search for each example in the test data
beam_search_results = [beam_search_decoder(sample, k=3) for sample in predicted_summaries]

# Convert sequences back to text and print examples
for i in range(10):  
    actual_summary = " ".join([target_tokenizer.index_word.get(word, "") for word in y_test[i] if word != 0])
    predicted_summaries_text = []
    for beam_result in beam_search_results[i]:
        predicted_summary = " ".join([target_tokenizer.index_word.get(word, "") for word in beam_result[0] if word != 0])
        predicted_summaries_text.append(predicted_summary)
    print(f"Example - Actual Summary: {actual_summary}")
    print(f"Example - Predicted Summaries: {predicted_summaries_text}\n")

C:\Users\sonal\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 274ms/step - accuracy: 0.0017 - loss: 8.5164 
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 310ms/step - accuracy: 0.0468 - loss: 8.5029
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 275ms/step - accuracy: 0.0444 - loss: 8.4440
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 271ms/step - accuracy: 0.0473 - loss: 8.5308
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 274ms/step - accuracy: 0.0425 - loss: 8.3054
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


C:\Users\sonal\AppData\Local\Temp\ipykernel_20964\2441848445.py:71: RuntimeWarning: overflow encountered in scalar multiply
  candidate = [seq + [j], score * -np.log(row[j])]


Example - Actual Summary: are the primary for the of and and models have in understanding these however the of a such as a should factors this scoping review aims to identify and the most factors as well as for of and
Example - Predicted Summaries: ['studies studies studies studies studies studies the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the', 'studies studies studies studies studies studies the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the', 'studies studies studies studies studies studies the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the of']

Example - Actual Summary: the of clinical is a before more natural language processing models that high accuracy for 1 experience a large of accuracy when to